In [12]:
import os
from urllib.parse import parse_qs
from IPython.display import Markdown
import csv

In [15]:
ROOT_PATH = "/exp/dafny_repair/"

In [81]:
query_string = os.getenv("QUERY_STRING")
row_id = None
if query_string:
    if "method" in parse_qs(query_string):
        row_id = int(parse_qs(query_string)["method"][0])
    if "results" in parse_qs(query_string):
        result_filepath = os.path.join(ROOT_PATH, parse_qs(query_string)["results"][0])
    if "assertions" in parse_qs(query_string):
        assertion_filepath = os.path.join(
            ROOT_PATH, parse_qs(query_string)["assertions"][0]
        )

In [80]:
with open(result_filepath, "r") as file:
    csv_reader = csv.DictReader(file)
    for index, row in enumerate(csv_reader):
        if row["Index"] == str(row_id):
            result = row
            break

with open(assertion_filepath, "r") as file:
    csv_reader = csv.DictReader(file)
    for index, row in enumerate(csv_reader):
        if index == row_id:
            assertion = row
            break

KeyError: 'Index'

In [66]:
def extract_dafny_functions(dafny_code, name):
    inside_function = False
    current_function = ""
    brace_count = 0

    lines = dafny_code.split("\n")

    for line in lines:
        if f"lemma {name}" in line or f"method {name}" in line:
            inside_function = True
            current_function += line + "\n"
            brace_count += line.count("{") - line.count("}")
        elif inside_function:
            current_function += line + "\n"
            brace_line_count = line.count("{") - line.count("}")
            brace_count += brace_line_count

            # hack to identify when a function finishes
            # {} if it is an empty function
            # otherwise expect } and with a brace count to 0 (since we are finishing the func)
            # and not an even number of braces on the line (the odd is the one to finish the func)
            if (
                brace_count == 0
                and "}" in line
                and (brace_line_count != 0 or "{}" in line)
            ):
                inside_function = False
                return current_function


def remove_warning(message):
    lines = message.split("\n")

    # Find the index of the first line containing an error
    error_index = next(
        (i for i, line in enumerate(lines) if "Error" in line), len(lines)
    )
    filtered_lines = lines[error_index:]
    return "\n".join(filtered_lines)

In [53]:
def display_in_markdown_codeblock(content):
    return Markdown(f"```\n{content}\n```")


def display_content_in_markdown_codeblock(file_path):
    with open(file_path, "r") as file:
        content = file.read()
    return display_in_markdown_codeblock(content)

In [52]:
Markdown(
    f'# Assertion : `{assertion["Assertion"][:-1]}` in {assertion["Original Method"]}'
)

# Assertion : `assert E == (iset s | Rand.Head(s) == (m % 2 == 1));` in SampleRecursiveHalf

In [57]:
Markdown(f"## Original Method")

## Original Method

In [56]:
base_file = assertion["Original File"]
with open(base_file, "r") as file:
    base_code = file.read()
og_method = extract_dafny_functions(base_code, assertion["Original Method"])

display_in_markdown_codeblock(og_method)

```
  lemma SampleRecursiveHalf(n: nat, m: nat)
    requires n >= 2
    ensures Rand.prob(iset s | Model.Sample(n)(s).value == m) == Rand.prob(iset s | Model.Sample(n / 2)(s).value == m / 2) / 2.0
  {
    var aOf: Rand.Bitstream -> nat := (s: Rand.Bitstream) => Model.Sample(n / 2)(s).value;
    var bOf: Rand.Bitstream -> bool := (s: Rand.Bitstream) => Monad.Coin(Model.Sample(n / 2)(s).rest).value;
    var A: iset<nat> := (iset x: nat | x == m / 2);
    var E: iset<Rand.Bitstream> := (iset s | m % 2 as nat == Helper.boolToNat(Monad.Coin(s).value));
    var f := (s: Rand.Bitstream) => Model.Sample(n / 2)(s).rest;

    var e1 := (iset s | Model.Sample(n / 2)(s).RestIn(E));
    var e2 := (iset s | Model.Sample(n / 2)(s).In(A));
    var e3 := (iset s | 2*aOf(s) + Helper.boolToNat(bOf(s)) == m);

    assert SplitEvent: e3 == e1 * e2 by {
      forall s ensures s in e3 <==> s in e1 && s in e2 {
        var a: nat := aOf(s);
        var b: nat := Helper.boolToNat(bOf(s));
        assert b < 2;
        calc {
          s in e3;
          2 * a + b == m;
          m == a * 2 + b;
          (a == m / 2) && (b == m % 2);
          s in e1 && s in e2;
        }
      }
    }

    assert Eq2: (iset s | aOf(s) == m / 2) == e2 by {
      forall s ensures aOf(s) == m / 2 <==> Model.Sample(n / 2)(s).value in A {
      }
    }

    assert Eq3: (iset s | aOf(s) == m / 2) == (iset s | Model.Sample(n / 2)(s).value == m / 2) by {
      forall s ensures aOf(s) == m / 2 <==> Model.Sample(n / 2)(s).value == m / 2 {
        assert aOf(s) == Model.Sample(n / 2)(s).value;
      }
    }

    assert Eq4: e1 == Measures.PreImage(f, E) by {
      forall s ensures Model.Sample(n / 2)(s).rest in E <==> f(s) in E {
      }
    }

    assert E in Rand.eventSpace && Rand.prob(E) == 0.5 by {
            Rand.CoinHasProbOneHalf(m % 2 == 1);
    }

    assert Indep: Rand.prob(e1 * e2) == Rand.prob(e1) * Rand.prob(e2) by {
      assert Measures.AreIndepEvents(Rand.eventSpace, Rand.prob, e1, e2) by {
        assert Independence.IsIndepFunction(Model.Sample(n / 2)) by {
          assert Independence.IsIndep(Model.Sample(n / 2)) by {
            SampleIsIndep(n / 2);
          }
          Independence.IsIndepImpliesIsIndepFunction(Model.Sample(n / 2));
        }
        assert E in Rand.eventSpace;
        assert Independence.IsIndepFunctionCondition(Model.Sample(n / 2), A, E);
      }
      Independence.AreIndepEventsConjunctElimination(e1, e2);
    }

    assert ProbE1: Rand.prob(e1) == 0.5 by {
      calc {
        0.5;
      ==
        Rand.prob(E);
      == { SampleIsMeasurePreserving(n / 2); }
        Rand.prob(Measures.PreImage(f, E));
      == { reveal Eq4; }
        Rand.prob(e1);
      }
    }

    calc {
      Rand.prob(iset s | Model.Sample(n)(s).value == m);
    == { SampleSetEquality(n, m); }
      Rand.prob(e3);
    == { reveal SplitEvent; }
      Rand.prob(e1 * e2);
    == { reveal Indep; }
      Rand.prob(e1) * Rand.prob(e2);
    == { reveal ProbE1; Helper.Congruence(Rand.prob(e1), 0.5, x => x * Rand.prob(e2)); }
      0.5 * Rand.prob(e2);
    ==
      Rand.prob(e2) / 2.0;
    == { reveal Eq2; }
      Rand.prob(iset s | aOf(s) == m / 2) / 2.0;
    == { reveal Eq3; }
      Rand.prob(iset s | Model.Sample(n / 2)(s).value == m / 2) / 2.0;
    }
  }

```

In [58]:
Markdown(f"## Error Message")

## Error Message

In [69]:
error_file = os.path.join(ROOT_PATH, result["Original Error Message File"])
with open(error_file, "r") as file:
    error_message = file.read()
display_in_markdown_codeblock(remove_warning(error_message))

```
/exp/test_projects/Dafny-VMC/src/Math/Analysis/Limits.dfy(6,7): Error: Duplicate module name: Limits [Related location] /exp/test_projects/Dafny-VMC/src/Math/Analysis/LimitIsAdditive_fix_2.dfy(6,7)
1 resolution/type errors detected in the_program

```

In [63]:
Markdown(f"## Assertion generated")

## Assertion generated

In [62]:
display_in_markdown_codeblock(result["Diff"])

```
assert 2.0 in upper;  // Adding this assertion
assert sqrt2 >= 1.0 && sqrt2 <= 2.0;  // Correcting the assertion
// the rest of the method
}
```

In [64]:
Markdown(f'## Result: {result["New Method Result"]}')

## Result: Errors

In [78]:
if (
    result["New Method Result"] == "Error"
    and result["New Method Error Message File"] != ""
):
    error_file = os.path.join(ROOT_PATH, result["New Method Error Message File"])
    with open(error_file, "r") as file:
        error_message = file.read()
    display_in_markdown_codeblock(remove_warning(error_message))

In [71]:
Markdown(f"## Prompt")

## Prompt

In [72]:
display_content_in_markdown_codeblock(result["Prompt File"])

```
<|im_start|>system
"You are a Dafny formal method expert.
You will be provided with a Dafny method indicated by the delimiter <method>
that does not verify because of an error indicated in between <error> tagsalong
along with its dependencies with the delimiter <dependencies>.
Your task is to insert an assertion in the <assertion> placeholder to make it verify."
<|im_end|><|im_start|>user
Can you fix this proof by inserting one assertion in the <assertion> placeholder? 
 <method>
lemma LemmaFlattenLengthLeMul<T>(s: seq<seq<T>>, j: int)
  requires forall i {{:trigger s[i]}} | 0 <= i < |s| :: |s[i]| <= j
  ensures |FlattenReverse(s)| <= |s| * j
  {{
    if |s| == 0 {{
    }} else {{
      LemmaFlattenLengthLeMul(s[..|s|-1], j);
      <assertion> Insert assertion here </assertion>
    }}
  }}
  # https://github.com/dafny-lang/libraries/pull/56/commits/4586789e1af32732224d7b7c31ccb55035ffac04
 </method><|im_end|><|im_start|>assistant
 lemma LemmaFlattenLengthLeMul<T>(s: seq<seq<T>>, j: int)
  requires forall i {{:trigger s[i]}} | 0 <= i < |s| :: |s[i]| <= j
  ensures |FlattenReverse(s)| <= |s| * j
  {{
    if |s| == 0 {{
    }} else {{
      LemmaFlattenLengthLeMul(s[..|s|-1], j);
+     assert |FlattenReverse(s[..|s|-1])| <= (|s|-1) * j;
    }}
  }}
  # https://github.com/dafny-lang/libraries/pull/56/commits/4586789e1af32732224d7b7c31ccb55035ffac04<|im_end|><|im_start|>user
Can you fix this proof by inserting one assertion in the <assertion> placeholder? 
 <method>
  lemma LemmaMaxOfConcat(xs: seq<int>, ys: seq<int>)
    requires 0 < |xs| && 0 < |ys|
    ensures Max(xs+ys) >= Max(xs)
    ensures Max(xs+ys) >= Max(ys)
    ensures forall i {:trigger i in [Max(xs + ys)]} :: i in xs + ys ==> Max(xs + ys) >= i
  {
    reveal Max();
    if |xs| == 1 {
    } else {
      <assertion> Insert assertion here </assertion>
      LemmaMaxOfConcat(xs[1..], ys);
    }
  }
 </method><|im_end|><|im_start|>assistant
   lemma LemmaMaxOfConcat(xs: seq<int>, ys: seq<int>)
    requires 0 < |xs| && 0 < |ys|
    ensures Max(xs+ys) >= Max(xs)
    ensures Max(xs+ys) >= Max(ys)
    ensures forall i {:trigger i in [Max(xs + ys)]} :: i in xs + ys ==> Max(xs + ys) >= i
  {
    reveal Max();
    if |xs| == 1 {
      assert  xs+ys == [xs[0]] + (xs[1..] + ys);
    } else {
      assert  xs+ys == [xs[0]] + (xs[1..] + ys);
      LemmaMaxOfConcat(xs[1..], ys);
    }
  }<|im_end|><|im_start|>user
Can you fix this proof by inserting one assertion in the <assertion> placeholder?
 <method>   lemma Sqrt2Exists() returns (sqrt2: real)
    ensures sqrt2 * sqrt2 == 2.0
  {
    var lower := iset x: real | x >= 0.0 && Square(x) < 2.0;
    var upper := iset x: real | x >= 0.0 && Square(x) > 2.0;
    forall x <- lower, y <- upper ensures x < y {
      if x >= y {
        calc {
          2.0;
        <
          Square(y);
        <= { RealArith.MulMonotonic(y, y, x); RealArith.MulMonotonic(x, y, x); }
          Square(x);
        <
          2.0;
        }
      }
    }
    assert 1.0 in lower;
        sqrt2 := Reals.DedekindCompleteness(lower, upper);
<assertion> Insert assertion here </assertion>
    assert 1.0 <= sqrt2 <= 2.0;
    assert Square(sqrt2) == 2.0 by {
      if sqrt2 * sqrt2 > 2.0 {
        var eps := (sqrt2 * sqrt2 - 2.0) / (2.0 * sqrt2);
        assert 0.0 < eps < 1.0;
        var y := sqrt2 - eps;
        assert y * y > 2.0 by {
          calc {
            y * y;
            sqrt2 * sqrt2 - 2.0 * sqrt2 * eps + eps * eps;
            sqrt2 * sqrt2 - (sqrt2 * sqrt2 - 2.0) + eps * eps;
            2.0 + eps * eps;
          >
            2.0;
          }
        }
        assert y in upper by {
          assert y >= 0.0;
          assert y * y > 2.0;
        }
        assert false;
      }
      if sqrt2 * sqrt2 < 2.0 {
        var eps := (2.0 - sqrt2 * sqrt2) / (2.0 * sqrt2 + 1.0);
        assert 0.0 < eps < 1.0;
        var y := sqrt2 + eps;
        assert y * y < 2.0 by {
          calc {
            y * y;
            sqrt2 * sqrt2 + 2.0 * sqrt2 * eps + eps * eps;
            sqrt2 * sqrt2 + (2.0 * sqrt2 + eps) * eps;
          < { RealArith.MulMonotonicStrict(eps, 2.0 * sqrt2 + eps, 2.0 * sqrt2 + 1.0); }
            sqrt2 * sqrt2 + (2.0 * sqrt2 + 1.0) * eps;
            sqrt2 * sqrt2 + (2.0 - sqrt2 * sqrt2);
            2.0;
          }
        }
        assert y in lower by {
          assert y >= 0.0;
          assert y * y < 2.0;
        }
        assert false;
      }
    }
  }
 </method><|im_end|><|im_start|>assistant
  lemma Sqrt2Exists() returns (sqrt2: real)
    ensures sqrt2 * sqrt2 == 2.0
  {
    var lower := iset x: real | x >= 0.0 && Square(x) < 2.0;
    var upper := iset x: real | x >= 0.0 && Square(x) > 2.0;
    forall x <- lower, y <- upper ensures x < y {
      if x >= y {
        calc {
          2.0;
        <
          Square(y);
        <= { RealArith.MulMonotonic(y, y, x); RealArith.MulMonotonic(x, y, x); }
          Square(x);
        <
          2.0;
        }
      }
    }
    assert 1.0 in lower;
        sqrt2 := Reals.DedekindCompleteness(lower, upper);
    assert 1.0 < sqrt2 < 2.0;  // Inserting this assertion to verify sqrt2's range before further processing.
    assert 1.0 <= sqrt2 <= 2.0;
    assert Square(sqrt2) == 2.0 by {
      if sqrt2 * sqrt2 > 2.0 {
        var eps := (sqrt2 * sqrt2 - 2.0) / (2.0 * sqrt2);
        assert 0.0 < eps < 1.0;
        var y := sqrt2 - eps;
        assert y * y > 2.0 by {
          calc {
            y * y;
            sqrt2 * sqrt2 - 2.0 * sqrt2 * eps + eps * eps;
            sqrt2 * sqrt2 - (sqrt2 * sqrt2 - 2.0) + eps * eps;
            2.0 + eps * eps;
          >
            2.0;
          }
        }
        assert y in upper by {
          assert y >= 0.0;
          assert y * y > 2.0;
        }
        assert false;
      }
      if sqrt2 * sqrt2 < 2.0 {
        var eps := (2.0 - sqrt2 * sqrt2) / (2.0 * sqrt2 + 1.0);
        assert 0.0 < eps < 1.0;
        var y := sqrt2 + eps;
        assert y * y < 2.0 by {
          calc {
            y * y;
            sqrt2 * sqrt2 + 2.0 * sqrt2 * eps + eps * eps;
            sqrt2 * sqrt2 + (2.0 * sqrt2 + eps) * eps;
          < { RealArith.MulMonotonicStrict(eps, 2.0 * sqrt2 + eps, 2.0 * sqrt2 + 1.0); }
            sqrt2 * sqrt2 + (2.0 * sqrt2 + 1.0) * eps;
            sqrt2 * sqrt2 + (2.0 - sqrt2 * sqrt2);
            2.0;
          }
        }
        assert y in lower by {
          assert y >= 0.0;
          assert y * y < 2.0;
        }
        assert false;
      }
    }
  }
  The added assertion `assert 1.0 < sqrt2 < 2.0;` makes sure that the value of sqrt2 lies within the expected range before processing it further. This helps in avoiding possible computational errors or anomalies in the later parts of the method.<|im_end|><|im_start|>user
This is the new error message that we get after the indicated change:
 <error>
/exp/test_projects/Dafny-VMC/src/Math/Analysis/Sqrt2Exists_fix_7.dfy(148,8): Error: a precondition for this call could not be proved
    |
148 |         sqrt2 := Reals.DedekindCompleteness(lower, upper);
    |                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

/exp/test_projects/Dafny-VMC/src/Math/Analysis/Sqrt2Exists_fix_7.dfy(43,13): Related location: this is the precondition that could not be proved
   |
43 |     requires upper != iset{}
   |              ^^^^^^^^^^^^^^^

/exp/test_projects/Dafny-VMC/src/Math/Analysis/Sqrt2Exists_fix_7.dfy(149,11): Error: assertion might not hold
    |
149 |     assert 1.0 < sqrt2 < 2.0;  // Inserting this assertion to verify sqrt2's range before further processing.
    |            ^^^^^^^^^^^

/exp/test_projects/Dafny-VMC/src/Math/Analysis/Sqrt2Exists_fix_7.dfy(149,17): Error: assertion might not hold
    |
149 |     assert 1.0 < sqrt2 < 2.0;  // Inserting this assertion to verify sqrt2's range before further processing.
    |                  ^^^^^^^^^^^


Dafny program verifier finished with 0 verified, 3 errors

 <\error><|im_end|>
```

In [73]:
Markdown(f"## New Method")

## New Method

In [74]:
base_file = assertion["New Method File"]
with open(base_file, "r") as file:
    base_code = file.read()
og_method = extract_dafny_functions(base_code, assertion["Original Method"])

display_in_markdown_codeblock(og_method)

FileNotFoundError: [Errno 2] No such file or directory: '/exp/test_projects/Dafny-VMC/src/Distributions/UniformPowerOfTwo/SampleRecursiveHalf_fix_70.dfy'